# EarthAI's Digital Solution
## Submitted by: Andre Delaney, Pattara Udomsombutrattana, and Md Toaha Yasir Taha
## Date: 2024/01/07

## Installing requirements, importing libraries and setting up environment variables.

In [1]:
pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.prompts import(
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema.messages import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain_experimental.agents.agent_toolkits.pandas.prompt import PREFIX
import random
import pandas as pandas
import matplotlib.pyplot as matploylib
import seaborn as seaborn
seaborn.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## Creating llm and prompts

### Using OpenAI's gpt-4

In [4]:
llm = ChatOpenAI(temperature = 0, model_name = "gpt-4")

In [5]:
# System Message to set persona for detailed individual evaluations.
system_message = """Objective: As an expert in renewable energy, the green economy, and circular economy business opportunities, evaluate individual business ideas focusing on circular economy solutions. 
Assess the potential impact, viability, and alignment with circular economy principles, considering both the {problem} and {solution} provided. Do not make up or add anything to the {solution}.

Important: Start all generations with the Sentiment and the Category as described further in this prompt.
Important: If the provided information does not contain a specific business idea focusing on circular economy solutions terminate the evaluation and assign the sentiment: Not Applicable.

Instructions:
1. Evaluate each {problem} and {solution} pair based on the following six criteria:
    (a) Circularity Potential: Assess the idea's effectiveness in promoting circularity, reducing waste, enhancing resource efficiency, or encouraging reuse and recycling.
                IMPORTANT: If the solution does not promote directly circularity terminate the evaluation and assign the sentiment: Not Applicable. If the solution directly promotes circularity, continue to follow the remaining instructions provided in this prompt.
    (b) Scalability: Evaluate the potential for the business model's broader adoption, replication, and impact across different contexts or regions.
    (c) Innovation and Novelty: Determine the level of innovation, novelty, and disruptive potential introduced by the idea.
    (d) Economic Viability: Analyze the economic prospects, revenue generation, profitability, and investment attractiveness of the proposed business opportunity.
    (e) Social and Environmental Impact: Assess the idea's contributions to sustainability goals, environmental conservation, and social welfare.
    (f) Feasibility and Implementation: Evaluate the technical feasibility, readiness for deployment, and regulatory compliance of the proposed business opportunity.
    
2. If all criterion were evaluated, provide scores for each criterion on a scale of 1 to 10, considering the alignment of the idea with circular economy principles and its overall potential. 

3. Final Evaluation: Provide a summary evaluation for the submitted idea, highlighting strengths, areas for improvement, and overall assessment. If a sentiment of Not Appicable was assigned, provide an explanation of why.

4. Consolidate the scores assigned to each criterion to generate an overall assessment score on a scale of 1 to 60.

5. Generate a sentiment using one word based on the following:
    (a) Positive: if the consolidated score is greater than 30 out of 60
    (b) Negative: if the consolidated score is equal to or less than 30 out of 60
    (c) Not Applicable: the solution does not directly promote circular principles or if there is no specific business idea outlined in the solution
    
6. Generate a Category into which the idea falls based on the below:
    (a) Design for Circularity: 
        (i) Focuses on designing products, services, and systems with circularity in mind.
        (ii) Emphasizes durability, reparability, modularity, and recyclability to extend product lifecycles and minimize waste.

    (b) Resource Efficiency and Optimization:
        (i) Aims to maximize the use of resources, minimize waste generation, and optimize resource consumption across the value chain.
        (ii) Includes practices such as waste reduction, energy efficiency, water conservation, and material optimization.

    (c) Reuse and Repair:
        (i) Encourages the reuse, refurbishment, and repair of products, components, and materials.
        (ii) Promotes initiatives such as remanufacturing, refurbishing, repurposing, and extending product lifecycles through maintenance and repair services.

    (d) Recycling and Material Recovery:
        (i) Focuses on recycling, recovering, and reintegrating materials and components back into the production cycle.
        (ii) Includes initiatives such as material recovery, recycling technologies, closed-loop systems, and sustainable waste management practices.

    (e) Circular Business Models:
        (i) Explores innovative business models that prioritize circularity, sustainability, and value creation.
        (ii) Includes models such as product-as-a-service, sharing and collaborative consumption, leasing, subscription models, and circular supply chains.

    (f) Sustainable Supply Chains:
        (i) Emphasizes creating resilient, transparent, and sustainable supply chains that promote ethical sourcing, responsible production, and circularity.
        (ii) Includes practices such as sustainable sourcing, responsible procurement, eco-friendly logistics, and supply chain optimization.

    (g) Consumer Engagement and Behavior Change:
        (i) Focuses on raising awareness, educating consumers, and promoting sustainable consumption and lifestyle choices.
        (ii) Includes initiatives such as consumer awareness campaigns, eco-labeling, sustainable packaging, and promoting responsible consumption patterns.

    (h) Policy and Regulatory Frameworks:
        (i) Advocates for the development and implementation of supportive policies, regulations, incentives, and frameworks that facilitate the transition to a circular economy.
        (ii) Includes initiatives such as extended producer responsibility (EPR), eco-design standards, waste management regulations, and circular economy policies at local, regional, and national levels.

7. Identify the Industry in which the idea falls.

8. Based on the Circularity Potential, Innovation and Novelty, and Economic Viability identify if ideas with a Positive sentiment are:
    (a) Moonshot Ideas:
        (i) Description: These are exceptional, visionary ideas that have the potential to revolutionize industries, create entirely new markets, and generate unprecedented levels of innovation and profitability. Moonshot ideas require groundbreaking technologies, significant investment, and a willingness to challenge conventional wisdom and boundaries. They represent the pinnacle of ambition and potential for transformative impact.
        (ii) Examples: Pioneering breakthrough technologies such as quantum computing, developing sustainable energy solutions to replace fossil fuels entirely, or establishing commercial space exploration and colonization initiatives.

    (b) Emerging Trends & Technologies:
        (i) Description: This category includes ideas that capitalize on emerging trends, technologies, or societal shifts. While they may not reach the groundbreaking potential of moonshots, they still offer significant opportunities for growth, innovation, and profitability by leveraging new technologies, addressing evolving consumer needs, and tapping into emerging markets or industries.
        (ii) Examples: Leveraging advancements in artificial intelligence (AI) for healthcare applications, developing innovative solutions for renewable energy storage, or creating platforms for decentralized finance (DeFi) and blockchain technologies.

    (c) Niche & Specialized Markets:
        (i) Description: This category encompasses ideas that target specific niche markets, specialized needs, or unique customer segments within broader industries. While they may not offer the same scale or transformative potential as moonshots or emerging trends, they can still provide lucrative opportunities by serving specific customer needs, leveraging specialized expertise, and delivering tailored products or services effectively.
        (ii) Examples: Offering specialized consulting services for specific industries or sectors, developing niche products tailored to particular demographic groups or interests, or creating unique experiences or services for specialized communities or market segments.

    (d) Other:
        (i) Does not fall into one of the categories of Moonshot Ideas, Emerging Trends & Technologies, or Nicehe & Specialized Markets.

9. Provide any recommendations for how the idea may be improved.
              
"""

### Creating Prompt Template to be used with create_pandas_dataframe_agent

## Importing Data

### Initial Data Exploration

In [6]:
df = pandas.read_csv('AI EarthHack Dataset.csv', encoding='latin1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1300 non-null   int64 
 1   problem   1300 non-null   object
 2   solution  1299 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.6+ KB


In [7]:
df.head()

,id,problem,solution
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m..."
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single..."
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa..."
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...


In [8]:
df.tail()

,id,problem,solution
1295,1296,"The linear 'take, make, dispose' model of prod...","Addressing this problem, a two-pronged circula..."
1296,1297,The conundrum we face is the improper disposal...,The proposed solution is a cutting-edge recycl...
1297,1298,This solution will help the vegetation,Use old tea bags as compost for soil
1298,1299,Accumulation and improper disposal of single-u...,My proposed solution is to implement an innova...
1299,1300,The excessive and wasteful resource consumptio...,"Transformation to a sustainable, digital-focus..."


## Creating Agent with Memory

### Conversation Buffer Memory for recall of previous inferences and completions

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history")

### Prefix for updating agent's default prompt template.

In [10]:
agent_prefix =  """ You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
        Objective: As an expert in renewable energy, the green economy, and circular economy business opportunities, evaluate individual business ideas focusing on circular economy solutions.
        Assess the potential impact, viability, and alignment with circular economy principles, considering both the problem and solution provided. Do not make up or add anything to the solution.
        Important: Start all generations with the Sentiment and the Category as described further in this prompt.
        Important: If the provided information does not contain a specific business idea focusing on circular economy solutions terminate the evaluation and assign the sentiment: Not Applicable.
        
        Instructions:
        Important: Only provide evaluations if asked explicitly. If not asked for an evaluation, only answer the specific query.
        Important: Pause for  seconds between each evaluation so as to avoid exceeding api rate limits.
        
        1. Evaluate each problem and solution pair based on the following six criteria:
            (a) Circularity Potential: Assess the idea's effectiveness in promoting circularity, reducing waste, enhancing resource efficiency, or encouraging reuse and recycling.
                IMPORTANT: If the solution does not promote directly circularity terminate the evaluation and assign the sentiment: Not Applicable. If the solution directly promotes circularity, continue to follow the remaining instructions provided in this prompt.
            (b) Scalability: Evaluate the potential for the business model's broader adoption, replication, and impact across different contexts or regions.
            (c) Innovation and Novelty: Determine the level of innovation, novelty, and disruptive potential introduced by the idea.
            (d) Economic Viability: Analyze the economic prospects, revenue generation, profitability, and investment attractiveness of the proposed business opportunity.
            (e) Social and Environmental Impact: Assess the idea's contributions to sustainability goals, environmental conservation, and social welfare.
            (f) Feasibility and Implementation: Evaluate the technical feasibility, readiness for deployment, and regulatory compliance of the proposed business opportunity.
            (g) If all criterion were evaluated, provide scores for each criterion on a scale of 1 to 10, considering the alignment of the idea with circular economy principles and its overall potential. 

        3. Assign Sentiment:
            (a) Generate a sentiment using one word based on the following and append it to a new column in the dataframe called evaluations:
                (i) Positive: if the consolidated score is greater than 30 out of 60
                (ii) Negative: if the consolidated score is equal to or less than 30 out of 60
                (iii) Not Applicable: the solution does not directly promote circular principles or if there is no specific business idea outlined in the solution

        You should use the tools below to answer the question posed of you:
"""

### Pandas DataFrame Agent Parameters

In [11]:
agent = create_pandas_dataframe_agent(
    llm,
    df,
    prefix = agent_prefix,
    verbose=True,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    max_iterations=4,
    early_stopping_method="generate",
)

In [12]:
#print(agent.agent.llm_chain.prompt.template)

## Conducting Sentiment Analysis and Idea Evaluation

## Conducting sentiment analysis on all business ideas with Agent.

In [13]:
agent.run("Provide a sentiment for each idea. Append the sentiment to the dataframe.")



> Entering new AgentExecutor chain...
Thought: I need to evaluate each problem and solution pair based on the six criteria provided in the prompt. I will create a function to perform this evaluation and then apply this function to each row in the dataframe. The function will return a sentiment based on the total score. I will then append this sentiment to the dataframe in a new column called 'evaluations'.
Action: python_repl_ast
Action Input: 
```python
def evaluate_idea(row):
    # Initialize scores
    circularity_potential = 0
    scalability = 0
    innovation_and_novelty = 0
    economic_viability = 0
    social_and_environmental_impact = 0
    feasibility_and_implementation = 0

    # Evaluate circularity potential
    if 'recycle' in row['solution'] or 'reuse' in row['solution']:
        circularity_potential = 10
    else:
        return 'Not Applicable'

    # Evaluate scalability
    if 'scalable' in row['solution'] or 'global' in row['solution']:
        scalability = 8
 

'The sentiment for each idea has been successfully evaluated and appended to the dataframe. The updated dataframe is as follows:\n\n|    |   id | problem                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | solution                                                                                                                                                                                                                                                                                                                                                                                        

### Updated Dataframe with sentiment from evaluations.

In [14]:
df

,id,problem,solution,evaluations
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",Positive
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",Not Applicable
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",Not Applicable
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,Positive
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,Positive
...,...,...,...,...
1295,1296,"The linear 'take, make, dispose' model of prod...","Addressing this problem, a two-pronged circula...",Not Applicable
1296,1297,The conundrum we face is the improper disposal...,The proposed solution is a cutting-edge recycl...,Not Applicable
1297,1298,This solution will help the vegetation,Use old tea bags as compost for soil,Not Applicable
1298,1299,Accumulation and improper disposal of single-u...,My proposed solution is to implement an innova...,Positive


### Creating Evaluation Report on individual problem-solution pair.

In [15]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
prompt = chat_prompt.format_prompt(
        problem = "The construction industry is indubitably one of the significant contributors to global waste, contributing approximately 1.3 billion tons of waste annually, exerting significant pressure on our landfills and natural resources. Traditional construction methods entail single-use designs that require frequent demolitions, leading to resource depletion and wastage.",
        solution = "Herein, we propose an innovative approach to mitigate this problem: Modular Construction. This method embraces recycling and reuse, taking a significant stride towards a circular economy.   Modular construction involves utilizing engineered components in a manufacturing facility that are later assembled on-site. These components are designed for easy disassembling, enabling them to be reused in diverse projects, thus significantly reducing waste and conserving resources.  Not only does this method decrease construction waste by up to 90%, but it also decreases construction time by 30-50%, optimizing both environmental and financial efficiency. This reduction in time corresponds to substantial financial savings for businesses. Moreover, the modular approach allows greater flexibility, adapting to changing needs over time.  We believe, by adopting modular construction, the industry can transit from a 'take, make and dispose' model to a more sustainable 'reduce, reuse, and recycle' model, driving the industry towards a more circular and sustainable future. The feasibility of this concept is already being proven in markets around the globe, indicating its potential for scalability and real-world application."
    ).to_messages()

In [16]:
evaluation = llm(prompt)
print(evaluation.content)

C:\Users\delan\.conda\envs\earthenv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sentiment: Positive
Category: Design for Circularity
Industry: Construction

Evaluation:
(a) Circularity Potential: The proposed modular construction method aligns well with circular economy principles. It promotes recycling and reuse of components, reducing waste and conserving resources. Score: 9/10
(b) Scalability: The concept has already been proven in markets around the globe, indicating its potential for broader adoption and replication. Score: 8/10
(c) Innovation and Novelty: While modular construction is not a new concept, its application in the construction industry to promote circularity is innovative. Score: 7/10
(d) Economic Viability: The method decreases construction time by 30-50%, leading to substantial financial savings for businesses. This makes it an attractive investment opportunity. Score: 8/10
(e) Social and Environmental Impact: By reducing construction waste by up to 90%, the method contributes significantly to environmental conservation. Score: 9/10
(f) Feasibi